In [35]:
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np
from keras.metrics import MeanSquaredError
from keras.optimizers import Adam
import tensorflow as tf 


In [38]:
# Custom objects for loading models due to errors
custom_objects = {
    'mse': MeanSquaredError()
}

# load the models 
face_classifier = cv2.CascadeClassifier(r"C:\Users\sebbe\Desktop\Skolsaker\Emotion_Detection_CNN\haarcascade_frontalface_default.xml")
emo_classifier = load_model(r"C:\Users\sebbe\Desktop\Skolsaker\Github\school_projects\Deep Learning\Kunskapskontroll 2\emo_model .h5")
gender_classifier = load_model(r"C:\Users\sebbe\Desktop\Skolsaker\Github\school_projects\Deep Learning\Kunskapskontroll 2\gender_model.h5")
age_classifier = load_model(r"C:\Users\sebbe\Desktop\Skolsaker\Github\school_projects\Deep Learning\Kunskapskontroll 2\age_model.h5", custom_objects=custom_objects)
ethnicity_classifier = load_model(r"C:\Users\sebbe\Desktop\Skolsaker\Github\school_projects\Deep Learning\Kunskapskontroll 2\ethnicity_model.h5")

# compile the models to remove errors 
emo_classifier.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
age_classifier.compile(optimizer='adam', loss='mse', metrics=['mean_absolute_error'])
gender_classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
ethnicity_classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

# Labels 
emotion_labels = ['Angry', 'Disgusted', 'Afraid', 'Happy', 'Neutral', 'Sad', 'Surprised']
names_ethnicity = ['White', 'Black', 'Asian', 'Indian', 'Other']
name_genders = ['Male', 'Female']

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    labels = []
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray)


    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 255), 2)
        roi_gray = gray[y:y + h, x:x + w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)

        if np.sum([roi_gray]) != 0:
            roi = roi_gray.astype('float') / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)

            # Make the predictions
            emo_prediction = emo_classifier.predict(roi)[0]
            gender_prediction = gender_classifier.predict(roi)[0]
            age_prediction = age_classifier.predict(roi)[0]
            ethnicity_prediction = ethnicity_classifier.predict(roi)[0]

            # identify emotion, gender, age and ethnicity
            emotion = emotion_labels[emo_prediction.argmax()]
            gender = name_genders[int(round(gender_prediction[0]))]

            # adding age span 
            # age_mean = int(age_prediction[0])
            # age_span = f"{max(0, age_mean - 2)}-{min(100, age_mean + 2)}"

            # If i want just age 
            age = int(age_prediction[0])


            ethnicity = names_ethnicity[ethnicity_prediction.argmax()]

            label = f"{emotion}, {ethnicity}, {gender}, {age}"

            # change text size and cetralize text
            text_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
            text_x = x + (w - text_size[0]) // 2
            text_y = y - 10

            # adding an edge on the text to make it easier to read 
            thickness = 2
            border_color = (0, 0, 0) 

            # adding 
            cv2.putText(frame, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, border_color, thickness+3)
            cv2.putText(frame, label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        else:
            cv2.putText(frame, 'No Faces', (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Emotion, Ethnicity Detector, Gender, Age', frame)

    # Tryck på Esc för att avsluta
    if cv2.waitKey(1) & 0xFF == 27:
        break
    # Klicka på "X"-knappen för att avsluta
    if cv2.getWindowProperty('Emotion, Ethnicity Detector, Gender, Age', cv2.WND_PROP_VISIBLE) < 1:
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━